In [1]:
import pandas as pd

In [26]:
barrios = pd.read_csv("barrios_bogota_localidades_limpio2.csv")
properties = pd.read_csv("properties.csv")

In [27]:
properties.head()


,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,deposito,porteria,zona_de_lavanderia,gas,parqueadero,precio,direccion,nombre,descripcion,barrio
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,0,24 hrs,No Tiene,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,1,24 hrs,No Tiene,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,0,24 hrs,No Tiene,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,0,24 hrs,No Tiene,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,1,24 hrs,No Tiene,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN


In [58]:

def limpiar_texto(s):
    if isinstance(s, str):
        s = s.strip().upper()
        # Normalizar a NFD para separar tildes y luego eliminar caracteres diacríticos
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        return s
    return s 

import unicodedata
import numpy as np #

properties['barrio_limpio'] = properties['barrio'].apply(limpiar_texto)
barrios['barrio_limpio'] = barrios['barrio'].apply(limpiar_texto) # Asegúrate que barrios['barrio'] exista

# Intentar el merge inicial con los barrios limpios
merged_temp = properties.merge(barrios[['barrio_limpio', 'localidad']],
                               left_on='barrio_limpio',
                               right_on='barrio_limpio',
                               how='left')

# Identificar los barrios que aún no tienen localidad
barrios_sin_localidad_df = merged_temp[merged_temp['localidad'].isnull()]
barrios_sin = barrios_sin_localidad_df['barrio_limpio'].unique().tolist()

# Tu lista de barrios de referencia (con localidad)
barrios_ref = barrios['barrio_limpio'].unique().tolist()

print(f"Número de barrios únicos sin localidad antes del fuzzing: {len(barrios_sin)}")
print(f"Número de barrios únicos de referencia: {len(barrios_ref)}")


Número de barrios únicos sin localidad antes del fuzzing: 32
Número de barrios únicos de referencia: 1678


In [59]:
from thefuzz import process, fuzz # Importa fuzz explícitamente

coincidencias = []

# Iterar sobre los barrios sin localidad
for b in barrios_sin:
    # Intenta con token_sort_ratio que es más robusto a reordenamientos y palabras extras
    match, score = process.extractOne(b, barrios_ref, scorer=fuzz.token_sort_ratio)
    # Considera un umbral un poco más bajo, ej. 80 para un primer intento
    if score > 80:
        coincidencias.append((b, match, score))

coincidencias_df = pd.DataFrame(coincidencias, columns=['barrio_original', 'barrio_coincidente', 'score'])
print("Top 20 coincidencias con score > 80 (Token Sort Ratio):")
print(coincidencias_df.sort_values(by='score', ascending=False).head(20))

Top 20 coincidencias con score > 80 (Token Sort Ratio):
            barrio_original          barrio_coincidente  score
10  CIUDAD SALITRE ORIENTAL  CIUDAD SALITRE NORORIENTAL     94
11             PATIO BONITO              PATIO BONITO I     92
9                 CHAPINERO                 CHAPINERITO     90
4              BOSA CENTRAL                 BOSA CENTRO     87
7               LA FLORESTA                  LAS FLORES     86
6                  VERBENAL                 EL VERBENAL     84
0                  EL PRADO                 EL PARADERO     84
2               LA ALHAMBRA                    ALHAMBRA     84
3              ARBORIZADORA           ARBORIZADORA BAJA     83
1                   TOBERIN                  EL TOBERIN     82
5                   SOSIEGO                  EL SOSIEGO     82
8                 COMUNEROS               LOS COMUNEROS     82
12            LOS ALCAZARES                   ALCAZARES     82


In [60]:
faltantes = merged[merged['localidad'].isnull()]
print(f"Barrios sin coincidencia: {len(faltantes)}")
print(faltantes['barrio'].unique()[:50])  # muestra los primeros 50 únicos


Barrios sin coincidencia: 304
['usaquen' 'el prado' 'toberin' 'castilla' 'capellania' 'americas'
 'la alhambra' 'engativa' 'suba' 'arborizadora' 'bosa occidental'
 'tibabuyes' 'tintal sur' 'bosa 61' 'apogeo' 'fontibon san pablo'
 'ciudad usme' 'el rincon' 'bosa central' 'modelia' 'tintal norte'
 'fontibon' 'sosiego' 'verbenal' 'boyaca real' 'chico lago' 'la floresta'
 'comuneros' 'chapinero' 'ciudad salitre oriental' 'patio bonito'
 'los alcazares']


In [66]:
import pandas as pd
import unicodedata
import numpy as np

# 1. Definir la función de limpieza (¡CRUCIAL para asegurar la llave de unión!)
def limpiar_texto(s):
    """Limpia el texto estandarizando a mayúsculas y eliminando tildes/caracteres especiales."""
    if isinstance(s, str):
        s = s.strip().upper()
        # Elimina caracteres diacríticos (tildes)
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        # Puedes añadir aquí pasos adicionales de reemplazo si encontraste errores comunes (ej. "SANTA FE" -> "SANTAFE")
        return s
    return s

# --- Carga de Datos ---
# Asegúrate de usar el delimitador correcto
df_properties = pd.read_csv('properties.csv')
df_barrios_ref = pd.read_csv('barrios_bogota_localidades_limpio2.csv')

# --- Preparación de las Llaves de Unión ---

# A. Limpieza en la base de Propiedades
df_properties['barrio_limpio'] = df_properties['barrio'].apply(limpiar_texto)

# B. Limpieza en la base de Referencia
df_barrios_ref['barrio_limpio'] = df_barrios_ref['Barrio'].apply(limpiar_texto)

# C. Eliminar duplicados en la base de referencia
# A veces, un barrio puede aparecer más de una vez (ej. en diferentes UPZ). 
# Mantendremos la primera instancia (o puedes decidir una lógica diferente si es necesario).
df_barrios_ref_unique = df_barrios_ref[['Localidad', 'UPZ', 'barrio_limpio']].drop_duplicates(subset=['barrio_limpio'])


# --- 2. EL COMANDO DE MERGE ---
# Realizar el Left Merge usando el campo limpio como llave
df_merged = df_properties.merge(
    df_barrios_ref_unique,
    left_on='barrio_limpio',      # Llave de la tabla de propiedades
    right_on='barrio_limpio',     # Llave de la tabla de referencia
    how='left'                    # Mantiene todas las filas de df_properties
)

print("¡Merge completado!")
print(f"Número total de propiedades: {len(df_merged)}")


¡Merge completado!
Número total de propiedades: 585


In [67]:
# Contar el número de propiedades sin localidad asignada
propiedades_sin_localidad = df_merged['Localidad'].isna().sum()

print(f"\nPropiedades a las que NO se pudo asignar Localidad: {propiedades_sin_localidad}")
print(f"Porcentaje de datos faltantes: {(propiedades_sin_localidad / len(df_merged) * 100):.2f}%")

# Si quieres ver los 10 nombres de barrio problemáticos más comunes que aún faltan:
barrios_faltantes_frecuentes = df_merged[df_merged['Localidad'].isna()]['barrio_limpio'].value_counts().head(10)
print("\nTop 10 barrios que aún no coinciden:")
print(barrios_faltantes_frecuentes)


Propiedades a las que NO se pudo asignar Localidad: 304
Porcentaje de datos faltantes: 51.97%

Top 10 barrios que aún no coinciden:
barrio_limpio
SUBA               32
CASTILLA           27
EL PRADO           24
TINTAL SUR         22
TIBABUYES          21
CIUDAD USME        21
LA ALHAMBRA        14
USAQUEN            14
ENGATIVA           13
BOSA OCCIDENTAL    12
Name: count, dtype: int64


In [62]:
import unicodedata
import re

def limpiar_barrio(texto):
    if isinstance(texto, str):
        # Quitar espacios adelante/atrás
        texto = texto.strip()
        # Pasar a mayúsculas
        texto = texto.upper()
        # Eliminar acentos
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
        # Reemplazar caracteres especiales y dobles espacios
        texto = re.sub(r'[^A-Z0-9\s]', '', texto)
        texto = re.sub(r'\s+', ' ', texto)
    return texto


In [63]:
# Normalizamos los nombres en ambos DataFrames
import unicodedata

def limpiar_texto(s):
    if isinstance(s, str):
        s = s.strip().upper()
        s = ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )  # elimina tildes
    return s

barrios['barrio'] = barrios['barrio'].apply(limpiar_texto)
properties['barrio'] = properties['barrio'].apply(limpiar_texto)

In [65]:
merged = properties.merge(barrios, on='barrio', how='left')
print(f"Barrios sin coincidencia: {merged['localidad'].isnull().sum()}")

Barrios sin coincidencia: 304


In [54]:
faltantes = merged[merged['localidad'].isnull()]
print("Algunos barrios sin coincidencia:")
print(faltantes['barrio'].unique()[:50])


Algunos barrios sin coincidencia:
['USAQUEN' 'EL PRADO' 'TOBERIN' 'CASTILLA' 'CAPELLANIA' 'AMERICAS'
 'LA ALHAMBRA' 'ENGATIVA' 'SUBA' 'ARBORIZADORA' 'BOSA OCCIDENTAL'
 'TIBABUYES' 'TINTAL SUR' 'BOSA 61' 'APOGEO' 'FONTIBON SAN PABLO'
 'CIUDAD USME' 'EL RINCON' 'BOSA CENTRAL' 'MODELIA' 'TINTAL NORTE'
 'FONTIBON' 'SOSIEGO' 'VERBENAL' 'BOYACA REAL' 'CHICO LAGO' 'LA FLORESTA'
 'COMUNEROS' 'CHAPINERO' 'CIUDAD SALITRE ORIENTAL' 'PATIO BONITO'
 'LOS ALCAZARES']


In [57]:
barrios_sin_localidad = merged[merged['localidad'].isnull()][['barrio']].drop_duplicates()
print(f"Barrios sin coincidencia: {len(barrios_sin_localidad)}")
barrios_sin_localidad.head(20)


Barrios sin coincidencia: 32


,barrio
1,usaquen
3,el prado
4,toberin
13,castilla
16,capellania
19,americas
28,la alhambra
31,engativa
33,suba
36,arborizadora


In [55]:
barrios['barrio'] = barrios['barrio'].str.strip().str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
properties['barrio'] = properties['barrio'].str.strip().str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
